# 5차시: 텐서플로우 2.x 활용 감성 분석 기초

## AI 맛보기 5주차: 2020. 08. 04. 20:00 ~ 22:00 (120분)
1. 도구 불러오기 및 버전 확인
1. 학습 데이터 다운로드: 스탠포드 AI 연구실 - 영화 리뷰 DB
1. 학습 데이터 살펴보기: 차원, 미리보기
1. 학습 데이터 준비: 불러오기
1. 학습 데이터 준비: 전처리
1. 학습 데이터 준비: 전처리 결과 확인
1. 워드 임베딩
1. 학습 모델 정의 및 생성
1. 학습
1. 학습 결과 확인
1. 예측: 가지고 놀기! 

#### 참고자료
- [파이썬 3 표준 문서](https://docs.python.org/3/index.html)
- [텐서플로우 텍스트 분류](https://www.tensorflow.org/tutorials/keras/text_classification)

### 1. 도구 불러오기 및 버전 확인

In [ ]:
# 도구 준비
import os
import shutil
import random
import math
import re
import string

import tensorflow as tf # 텐서플로우
import matplotlib.pyplot as plt # 시각화 도구
%matplotlib inline
import matplotlib.font_manager as fm
import numpy as np

In [ ]:
print(f'Tensorflow 버전을 확인합니다: {tf.__version__}')

### 2. 학습 데이터 다운로드: (aclImdb) 영화 리뷰 DB

In [ ]:
# 데이터 다운로드
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
# 데이터 확인
print(f'데이터 압축 디렉터리 내부: {os.listdir(dataset_dir)}')

In [ ]:
# 학습 데이터 구조 확인
train_dir = os.path.join(dataset_dir, 'train')
print(f'학습 데이터 디렉터리 내부: {os.listdir(train_dir)}')

### 3. 학습 데이터 살펴보기: 미리보기

In [ ]:
# 긍정 학습 데이터 파일 확인
pos_train_dir = os.path.join(train_dir, 'pos')
pos_train_files = os.listdir(pos_train_dir)
neg_train_dir = os.path.join(train_dir, 'neg')
neg_train_files = os.listdir(neg_train_dir)
print('긍정 파일: ', end='')
for _ in range(0, 10):
    print(random.choice(pos_train_files), end=' ')
print()
print('부정 파일: ', end='')
for _ in range(0, 10):
    print(random.choice(neg_train_files), end=' ')
print()

In [ ]:
# 학습 데이터 내용 확인
## https://translate.google.co.kr/
## https://papago.naver.com/
sample_file = os.path.join(train_dir, 'pos', '5667_10.txt')
with open(sample_file, 'r') as f:
    print(f.read())

### 4. 학습 데이터 준비: 불러오기

In [ ]:
# 데이터 정리
print('''학습을 위한 데이터는 이런 구조가 되어야 합니다.
train/
....pos/
........file1.txt
........file2.txt
....neg/
........file1.txt
........file2.txt''')
remove_dir = os.path.join(train_dir, 'unsup')
print(f'불필요한 데이터 파일을 정리합니다. {remove_dir}')
shutil.rmtree(remove_dir)

In [ ]:
# 데이터 불러오기
batch_size = 32
validation_split = 0.2
seed = 20200804

print('학습 데이터 세트를 불러옵니다.')
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        train_dir,
        label_mode='binary',
        batch_size=batch_size, 
        validation_split=validation_split, 
        subset='training',
        seed=seed)

In [ ]:
# 불러온 데이터 확인
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

In [ ]:
print('레이블 번호 - 이름 확인!')
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

In [ ]:
print('검증 데이터 세트를 불러옵니다.')
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        train_dir, 
        label_mode='binary',
        batch_size=batch_size,
        validation_split=validation_split, 
        subset='validation', 
        seed=seed)

### 5. 학습 데이터 준비: 전처리

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    stripped_html = tf.strings.regex_replace(stripped_html, 
                                             '(?:^|\W)(?:the|a|an|be|was|am)(?:$|\W)', '')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation), '')

In [ ]:
max_features = 1000
sequence_length = 50

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        standardize=custom_standardization,
        max_tokens=max_features,
        output_mode='int',
        output_sequence_length=sequence_length)

In [ ]:
print('테스트 데이터와 레이블을 분리합니다.')
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [ ]:
print('학습 완료 후 사용할 테스트 세트를 불러옵니다.')
test_dir = os.path.join(dataset_dir, 'test')
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        test_dir,
        label_mode='binary',
        batch_size=batch_size)

### 6. 학습 데이터 준비: 전처리 결과 확인

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
print('테스트 데이터를 확인합니다.')
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[int(first_label)])
print("Vectorized review", vectorize_text(tf.expand_dims(first_review, -1), first_label))

In [ ]:
print('각 특징 벡터의 원소 값을 확인할 수 있습니다.')
voc_size = len(vectorize_layer.get_vocabulary())
print(f'Vocabulary size: {voc_size}')
base = 0
cnt = 10
if voc_size - cnt < base:
    print(f'{voc_size-cnt} 보다는 작은 수를 입력해야 합니다.') 
else:
    for i in range(base, base+cnt):
        print(f'{i:4d} ---> {vectorize_layer.get_vocabulary()[i]}')

In [ ]:
print('학습, 검증, 테스트 데이터를 모두 벡터화 합니다.')
vec_train_ds = raw_train_ds.map(vectorize_text)
vec_val_ds = raw_val_ds.map(vectorize_text)
vec_test_ds = raw_test_ds.map(vectorize_text)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

print('데이터 입력부를 최적화합니다.')
train_ds = vec_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = vec_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = vec_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 7. 워드 임베딩

In [ ]:
embedding_dim = 4

embedding_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(voc_size, embedding_dim),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(embedding_dim, activation='relu'),
        tf.keras.layers.Dense(1)
])

embedding_model.summary()

In [ ]:
embedding_model.compile(optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy'])

history = embedding_model.fit(
        train_ds,
        epochs=10,
        validation_data=val_ds)

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for i, word in enumerate(vectorize_layer.get_vocabulary()[2:], start=2):
    vec = embedding_model.layers[0].get_weights()[0][i] # skip 0, it's padding.
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

### 8. 학습 모델 정의 및 생성

In [ ]:
print('모델을 정의합니다.')
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(voc_size, embedding_dim, mask_zero=True),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(1)])
model.summary()

In [ ]:
print('모델을 준비합니다.')
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### 9. 학습 

In [ ]:
epochs = 10
history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs)

### 10. 학습 결과 확인

In [ ]:
print('모델 성능을 테스트합니다.')
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
history_dict = history.history
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

fig1 = plt.figure(figsize=(6, 10))
ax = fig1.add_subplot(2, 1, 1)
ax.plot(epochs, loss, 'bo', label='Training loss')
ax.plot(epochs, val_loss, 'b', label='Validation loss')
ax.set_ylim((0, math.ceil(max(max(loss), max(val_loss)))))
ax.set_title('Training and validation loss', fontsize=12)
ax.set_xlabel('Epochs', fontsize=10)
ax.set_ylabel('Loss', fontsize=10)
ax.legend()

ax = fig1.add_subplot(2, 1, 2)
ax.plot(epochs, acc, 'bo', label='Training acc')
ax.plot(epochs, val_acc, 'b', label='Validation acc')
ax.set_ylim((0, math.ceil(max(max(acc), max(val_acc)))))
ax.set_title('Training and validation accuracy', fontsize=12)
ax.set_xlabel('Epochs', fontsize=10)
ax.set_ylabel('Accuracy', fontsize=10)
ax.legend()

### 11. 예측: 가지고 놀기! 

In [ ]:
print('모델 출력')
export_model = tf.keras.Sequential([
    vectorize_layer,
    model,
    tf.keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

In [ ]:
# user_str = input('테스트 입력: ').strip()
# predicted = export_model.predict(tf.expand_dims(user_str, -1))[0][0]
# print(f'예측 결과: {raw_train_ds.class_names[int(round(predicted))]} ({predicted:0.4f})')

### 12. 모델 개선

In [ ]:
# model = tf.keras.Sequential([
#                 tf.keras.layers.Embedding(voc_size, embedding_dim, mask_zero=True),
#                 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
#                 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#                 tf.keras.layers.Dense(64, activation='relu'),
#                 tf.keras.layers.Dropout(0.5),
#                 tf.keras.layers.Dense(1)])